In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer
from 

tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=True)

df = pd.read_json("../data/emotion_dataset.json", orient="records", lines=True)
preprocessing = mean
